In [15]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [16]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [17]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [18]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7720


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""04/04/2025""","""4176""","""27 24 32 30 35""","""2427303235""","[24, 27, … 35]",24,27,30,32,35,"""24""","""27""","""30""","""32""","""35""",null,null,true,24,"""03_03_02_03""",3
"""03/04/2025""","""4175""","""11 06 05 20 12""","""0506111220""","[5, 6, … 20]",5,6,11,12,20,"""05""","""06""","""11""","""12""","""20""","""24""","""35""",true,16,"""01_05_01_08""",19
"""02/04/2025""","""4174""","""26 14 24 11 17""","""1114172426""","[11, 14, … 26]",11,14,17,24,26,"""11""","""14""","""17""","""24""","""26""","""05""","""20""",true,24,"""03_03_07_02""",-13
"""01/04/2025""","""4173""","""12 25 33 13 08""","""0812132533""","[8, 12, … 33]",8,12,13,25,33,"""08""","""12""","""13""","""25""","""33""","""11""","""26""",false,0,"""04_01_12_08""",-29
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""33""",false,0,"""09_06_04_08""",5
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""34""",true,2,"""02_10_14_06""",-8
"""29/03/2025""","""4170""","""23 06 04 09 10""","""0406091023""","[4, 6, … 23]",4,6,9,10,23,"""04""","""06""","""09""","""10""","""23""","""03""","""35""",false,0,"""02_03_01_13""",17
"""28/03/2025""","""4169""","""20 10 01 09 29""","""0109102029""","[1, 9, … 29]",1,9,10,20,29,"""01""","""09""","""10""","""20""","""29""","""04""","""23""",true,16,"""08_01_10_09""",-26
"""27/03/2025""","""4168""","""27 35 05 32 08""","""0508273235""","[5, 8, … 35]",5,8,27,32,35,"""05""","""08""","""27""","""32""","""35""","""01""","""29""",true,16,"""03_19_05_03""",42


In [19]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str):
  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  # chart_unijump = df_jumps_map.hvplot.bar(
  #   x='jumps_map',
  #   y='count'
  # )

  display(df_viz)
  display(chart)
  display(df_10_last)
  display(df_jumps_map)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list()
  return lst_post_str_b, lst_jumps_map

In [20]:
possible_b1, exclude_jump_maps_b1 = plot_post_col(col='str_b1', post_col='post_str_b1', test_value="05")

post_str_b1,count,percent
str,u32,f64
"""02""",90,13.333333
"""01""",89,13.185185
"""03""",70,10.37037
"""05""",68,10.074074
"""04""",63,9.333333
…,…,…
"""19""",1,0.148148
"""25""",1,0.148148
"""28""",1,0.148148


:Overlay
   .Bars.I  :Bars   [post_str_b1]   (percent)
   .HLine.I :HLine   [x,y]

fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""03/04/2025""","""4175""","""11 06 05 20 12""","""0506111220""","[5, 6, … 20]",5,6,11,12,20,"""05""","""06""","""11""","""12""","""20""","""24""","""35""",true,16,"""01_05_01_08""",19
"""27/03/2025""","""4168""","""27 35 05 32 08""","""0508273235""","[5, 8, … 35]",5,8,27,32,35,"""05""","""08""","""27""","""32""","""35""","""01""","""29""",true,16,"""03_19_05_03""",42
"""09/02/2025""","""4122""","""05 21 20 35 29""","""0520212935""","[5, 20, … 35]",5,20,21,29,35,"""05""","""20""","""21""","""29""","""35""","""07""","""32""",false,0,"""15_01_08_06""",-30
"""10/01/2025""","""4092""","""06 14 05 13 20""","""0506131420""","[5, 6, … 20]",5,6,13,14,20,"""05""","""06""","""13""","""14""","""20""","""02""","""27""",false,0,"""01_07_01_06""",23
"""09/01/2025""","""4091""","""14 32 17 05 06""","""0506141732""","[5, 6, … 32]",5,6,14,17,32,"""05""","""06""","""14""","""17""","""32""","""05""","""20""",false,0,"""01_08_03_15""",29
"""25/12/2024""","""4076""","""14 21 18 28 05""","""0514182128""","[5, 14, … 28]",5,14,18,21,28,"""05""","""14""","""18""","""21""","""28""","""10""","""32""",true,16,"""09_04_03_07""",1
"""22/12/2024""","""4073""","""19 20 35 07 05""","""0507192035""","[5, 7, … 35]",5,7,19,20,35,"""05""","""07""","""19""","""20""","""35""","""06""","""32""",false,0,"""02_12_01_15""",46
"""21/12/2024""","""4072""","""17 34 05 28 29""","""0517282934""","[5, 17, … 34]",5,17,28,29,34,"""05""","""17""","""28""","""29""","""34""","""05""","""35""",false,0,"""12_11_01_05""",23
"""19/12/2024""","""4070""","""34 05 22 18 30""","""0518223034""","[5, 18, … 34]",5,18,22,30,34,"""05""","""18""","""22""","""30""","""34""","""07""","""27""",false,0,"""13_04_08_04""",-21


jumps_map,count
str,u32
"""05_01_09_03""",3
"""02_06_10_05""",2
"""05_08_04_13""",2
"""04_04_17_05""",2
"""02_06_05_01""",2
…,…
"""02_03_08_13""",1
"""05_15_01_03""",1
"""01_13_03_01""",1


In [21]:
possible_blast, exclude_jump_maps_blast = plot_post_col(col='str_b5', post_col='post_str_blast', test_value="35")

post_str_blast,count,percent
str,u32,f64
"""35""",164,14.642857
"""34""",160,14.285714
"""32""",121,10.803571
"""33""",121,10.803571
"""31""",91,8.125
…,…,…
"""14""",2,0.178571
"""13""",2,0.178571
"""17""",2,0.178571


:Overlay
   .Bars.I  :Bars   [post_str_blast]   (percent)
   .HLine.I :HLine   [x,y]

fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""34""",true,2,"""02_10_14_06""",-8
"""27/03/2025""","""4168""","""27 35 05 32 08""","""0508273235""","[5, 8, … 35]",5,8,27,32,35,"""05""","""08""","""27""","""32""","""35""","""01""","""29""",true,16,"""03_19_05_03""",42
"""25/03/2025""","""4166""","""29 06 35 22 04""","""0406222935""","[4, 6, … 35]",4,6,22,29,35,"""04""","""06""","""22""","""29""","""35""","""06""","""34""",true,16,"""02_16_07_06""",31
"""14/03/2025""","""4155""","""35 11 03 07 15""","""0307111535""","[3, 7, … 35]",3,7,11,15,35,"""03""","""07""","""11""","""15""","""35""","""02""","""27""",true,26,"""04_04_04_20""",16
"""12/03/2025""","""4153""","""28 14 35 03 15""","""0314152835""","[3, 14, … 35]",3,14,15,28,35,"""03""","""14""","""15""","""28""","""35""","""07""","""30""",false,0,"""11_01_13_07""",-40
"""08/03/2025""","""4149""","""02 35 30 32 16""","""0216303235""","[2, 16, … 35]",2,16,30,32,35,"""02""","""16""","""30""","""32""","""35""","""02""","""31""",true,8,"""14_14_02_03""",25
"""06/03/2025""","""4147""","""30 17 13 21 35""","""1317213035""","[13, 17, … 35]",13,17,21,30,35,"""13""","""17""","""21""","""30""","""35""","""03""","""30""",true,8,"""04_04_09_05""",-14
"""09/02/2025""","""4122""","""05 21 20 35 29""","""0520212935""","[5, 20, … 35]",5,20,21,29,35,"""05""","""20""","""21""","""29""","""35""","""07""","""32""",false,0,"""15_01_08_06""",-30
"""07/02/2025""","""4120""","""08 35 23 28 17""","""0817232835""","[8, 17, … 35]",8,17,23,28,35,"""08""","""17""","""23""","""28""","""35""","""01""","""31""",false,0,"""09_06_05_07""",1


jumps_map,count
str,u32
"""01_06_14_02""",3
"""02_02_16_13""",2
"""23_03_01_04""",2
"""10_07_02_05""",2
"""06_06_03_16""",2
…,…
"""13_02_16_03""",1
"""03_01_12_18""",1
"""18_05_05_04""",1


In [22]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [23]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in combos:
  cols = []
  str_row = " ".join([str(r).zfill(2) for r in row])
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

df_Orows = pl.DataFrame(
  lst,
  schema=[
    # 'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(df_Orows)
print(f'{TOTAL_LINES=}')
display(df_Orows.head(10))

TOTAL_LINES=324632


id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102030405""","[1, 2, … 5]",1,2,3,4,5,"""01""","""02""","""03""","""04""","""05""",null,null,true,8,"""01_01_01_01""",0
"""0102030406""","[1, 2, … 6]",1,2,3,4,6,"""01""","""02""","""03""","""04""","""06""","""01""","""05""",true,8,"""01_01_01_02""",1
"""0102030407""","[1, 2, … 7]",1,2,3,4,7,"""01""","""02""","""03""","""04""","""07""","""01""","""06""",true,8,"""01_01_01_03""",2
"""0102030408""","[1, 2, … 8]",1,2,3,4,8,"""01""","""02""","""03""","""04""","""08""","""01""","""07""",true,8,"""01_01_01_04""",3
"""0102030409""","[1, 2, … 9]",1,2,3,4,9,"""01""","""02""","""03""","""04""","""09""","""01""","""08""",true,8,"""01_01_01_05""",4
"""0102030410""","[1, 2, … 10]",1,2,3,4,10,"""01""","""02""","""03""","""04""","""10""","""01""","""09""",true,8,"""01_01_01_06""",5
"""0102030411""","[1, 2, … 11]",1,2,3,4,11,"""01""","""02""","""03""","""04""","""11""","""01""","""10""",true,8,"""01_01_01_07""",6
"""0102030412""","[1, 2, … 12]",1,2,3,4,12,"""01""","""02""","""03""","""04""","""12""","""01""","""11""",true,8,"""01_01_01_08""",7
"""0102030413""","[1, 2, … 13]",1,2,3,4,13,"""01""","""02""","""03""","""04""","""13""","""01""","""12""",true,8,"""01_01_01_09""",8


In [24]:
df_Orows.filter(is_rare_combination=False)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040513""","[1, 2, … 13]",1,2,4,5,13,"""01""","""02""","""04""","""05""","""13""","""01""","""12""",false,0,"""01_02_01_08""",10
"""0102040523""","[1, 2, … 23]",1,2,4,5,23,"""01""","""02""","""04""","""05""","""23""","""01""","""22""",false,0,"""01_02_01_18""",20
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2829313435""","[28, 29, … 35]",28,29,31,34,35,"""28""","""29""","""31""","""34""","""35""","""28""","""35""",false,0,"""01_02_03_01""",-3
"""2830313334""","[28, 30, … 34]",28,30,31,33,34,"""28""","""30""","""31""","""33""","""34""","""28""","""35""",false,0,"""02_01_02_01""",-4
"""2830313435""","[28, 30, … 35]",28,30,31,34,35,"""28""","""30""","""31""","""34""","""35""","""28""","""35""",false,0,"""02_01_03_01""",-7


In [25]:
# possible_b1, exclude_jump_maps_b1

df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_blast),
  pl.col("str_b5").is_in(possible_blast),
  pl.col("is_rare_combination") == False
)

df_possible

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
"""0102040533""","[1, 2, … 33]",1,2,4,5,33,"""01""","""02""","""04""","""05""","""33""","""01""","""32""",false,0,"""01_02_01_28""",30
"""0102040534""","[1, 2, … 34]",1,2,4,5,34,"""01""","""02""","""04""","""05""","""34""","""01""","""33""",false,0,"""01_02_01_29""",31
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1029313435""","[10, 29, … 35]",10,29,31,34,35,"""10""","""29""","""31""","""34""","""35""","""10""","""35""",false,0,"""19_02_03_01""",-21
"""1029323335""","[10, 29, … 35]",10,29,32,33,35,"""10""","""29""","""32""","""33""","""35""","""10""","""34""",false,0,"""19_03_01_02""",-11
"""1029323435""","[10, 29, … 35]",10,29,32,34,35,"""10""","""29""","""32""","""34""","""35""","""10""","""35""",false,0,"""19_03_02_01""",-15


In [27]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0315182528""","[3, 15, … 28]",3,15,18,25,28,"""03""","""15""","""18""","""25""","""28""","""03""","""27""",false,0,"""12_03_07_03""",-21
